the mothod of label converting is optimised # finish

In [187]:
import numpy as np
import pandas as pd
import torch
import os

# convert sequence to matrix
def one_hot(sequence):
    nucl_list = ['A', 'T', 'C', 'G']
    nucl_dic = dict((int, nucl) for nucl, int in enumerate(nucl_list))
    _seq_data = [nucl_dic[base] for base in sequence]
    result = np.zeros((len(sequence), 4))
    for _, base in enumerate(_seq_data):
        result[_, base] = 1
    return(result)

# convert label to tensor
def label2tensor(labels):
    # open host info file
    host_info = pd.read_csv('/home/ouconstand/data/Virus_Host/non_homo_host_info.csv')
    host_dict = host_info[['Representation', 'HostClass']].drop_duplicates().set_index('Representation').to_dict(orient='dict')['HostClass']
    
    # convert accession to class taxon
    labels = [host_dict[label] for label in labels]
    
    labels_dict = dict((key, value) for value, key in enumerate(list(set(labels))))
    labels = [labels_dict[i] for i in labels]
    result = np.zeros((len(labels), len(set(labels))))
    
    for _, base in enumerate(labels):
        result[_, base] = 1
    # convert result to tensor
    result = torch.tensor(result).to(torch.float32)
    
    return result

# cut off sequence to 2k segment
def seq2onehot(sequence_dir):
    
    seq_label = [] # the list is including seq corresponding accession
    seq_data = [] # a sequence is converted to a list of 2k sub-seq
    
    for file in os.listdir(sequence_dir):
        
        id = file
        with open(sequence_dir + file, 'r') as f:
            seq = f.read()
            
        new_record = []
            
        for i in range(0, len(seq), 500):
            if (i + 2000) > len(seq):
                new_seq = seq[-2000:]
                new_record.append(new_seq)
                break
            new_seq = seq[i:i+2000]
            new_record.append(new_seq)
        
        seq_data.append(torch.tensor(np.array([one_hot(segment) for segment in new_record], dtype=np.float32)))

        seq_label.append(id)

    seq_label = label2tensor(seq_label)
        
    return seq_label, seq_data

In [209]:
data_path = '/home/ouconstand/data/Virus_Host/0309_seqdata/'

In [216]:
labels, train_data = seq2onehot(data_path)

In [190]:
train_data[0].shape

torch.Size([56, 2000, 4])

In [191]:
for i in range(4):
    print(labels[i].shape)

torch.Size([2])
torch.Size([2])
torch.Size([2])
torch.Size([2])


In [192]:
for i in range(4):
    print(train_data[i].shape)

torch.Size([56, 2000, 4])
torch.Size([56, 2000, 4])
torch.Size([54, 2000, 4])
torch.Size([50, 2000, 4])


In [281]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # three convolutional layers and a maxpool layer
        self.convs_layers = nn.ModuleList(
            nn.Conv1d(in_channels=4, out_channels=256, kernel_size=3),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=7),
        )
        # it is not necessary to build a softmax layer because of the loss function including a softmax layer
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc = nn.Linear(256, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self, x):
        x = x.permute(0,2,1)
        x = [F.relu(conv(x).sequeeze(3)) for conv in self.convs_layers]
        x = []
        # x = torch.cat(x, 1)
        # x = x.squeeze()
        # x = self.fc(x)
        # x = self.relu(x)
        
        return x

In [282]:
# input (batch_size, seq_length, feature_num)
model = MyModel()
print(model)

MyModel(
  (convs_pool_layers): Sequential(
    (0): Conv1d(4, 256, kernel_size=(3,), stride=(1,))
    (1): Conv1d(256, 256, kernel_size=(5,), stride=(1,))
    (2): Conv1d(256, 256, kernel_size=(7,), stride=(1,))
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=256, out_features=512, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)


In [283]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [284]:
for i in range(len(train_data)):
    print(model(train_data[i]).shape)
    break

AttributeError: 'list' object has no attribute 'shape'

In [302]:
x = model(train_data[0])


In [308]:
x[0].shape[0]

256

In [227]:
batch_size = 64 # the count of sample is 64

def train_model(data, label, model, loss_fn, optimizer):
    size = len(data)
    model.train()
    
    # compute prediction error
    pred = model(data)
    loss = loss_fn(pred, label)

    # back propafation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_size % 64 == 0:
        loss, current = loss.item(), (batch_size+1) *len(data)
        print(f'loss:{loss:>7f} [{current:>5d}/{size:>5d}]')


In [228]:
epochs = 6
for t in range(epochs):
    print(f'Epoch{t+1}\n-------------------------------')
    for i in range(t*64+64):
        train_model(train_data[i], labels, model, loss_fn, optimizer)

Epoch1
-------------------------------


ValueError: Expected input batch_size (55664) to match target batch_size (499).